In [1]:
# Required imports
import matplotlib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score
import re
# from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn import model_selection, naive_bayes, svm

In [2]:
# cleaning data
def clean_post(post):
    post = post.lower()
    post = re.sub(r"\n", " ", post)
    post = re.sub("[\<\[].*?[\>\]]", " ", post)
    post = re.sub(r"[^a-z ]", " ", post)
    post = re.sub(r"\b\w{1,3}\b", " ", post)
    return " ".join([x for x in post.split() if x not in stop_words])

In [3]:
# Different techniques for tackling class imbalance
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE

def balance_data(x, y, _type):
    if _type == 0:
        ros = RandomOverSampler(random_state=42)
        return ros.fit_resample(x, y)
    elif _type == 1:
        rus = RandomUnderSampler(random_state=42, replacement=True)
        return rus.fit_resample(x, y)
    elif _type == 2:
        smote = SMOTE()
        return smote.fit_resample(x, y)
    elif _type == 3:
        nm = NearMiss()
        return nm.fit_resample(x, y)
    elif _type == 6:
        tl = TomekLinks()
        return tl.fit_resample(x, y)
    return x, y
    # Another technique is penalizing the algo with class_weight=balanced, using stratified cross validation

In [4]:
# Load data
data = pd.read_csv('/home2/tgv2002/reddit_mental_health_dataset/reddit_dataset.csv')
data = shuffle(data)

# Class split stats
print(data.groupby(['mental_disorder'])[['mental_disorder']].describe())
x = data['post'].apply(lambda post: clean_post(post))

# Vectorizing text data
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(x)
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(X_counts)

                mental_disorder                              
                          count unique            top    freq
mental_disorder                                              
EDAnonymous               14577      1    EDAnonymous   14577
addiction                  7641      1      addiction    7641
adhd                      45631      1           adhd   45631
alcoholism                 5911      1     alcoholism    5911
anxiety                   57671      1        anxiety   57671
autism                     8869      1         autism    8869
bipolarreddit              5780      1  bipolarreddit    5780
bpd                       24294      1            bpd   24294
depression               117331      1     depression  117331
healthanxiety              8648      1  healthanxiety    8648
lonely                    23635      1         lonely   23635
ptsd                       8643      1           ptsd    8643
schizophrenia              8712      1  schizophrenia    8712
socialan

In [5]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(np.array(data['mental_disorder']))
# y = to_categorical(y1)

# 60-20-20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=321)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5, random_state=321)

In [6]:
def get_metrics(y_true, y_pred):
    result1 = classification_report(y_true, y_pred)
    print('Classification Report: ', result1)
    result2 = accuracy_score(y_true, y_pred)
    print('Accuracy: ', result2, "\n\n")

In [ ]:
import gc

# Creating the model and checking it for various undersampled cases
X_tr, y_tr = X_train, y_train

for _type in [1, 3, 6, -1, 0, 2]:
    print('#'*110)
    print()
    if _type == -1:
        print('Without any undersampling/oversampling')
    else:
        print(f'With sampling type: {_type}')
    print()
    print()
    model = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    X_train, y_train = balance_data(X_tr, y_tr, _type)
    model.fit(X_train, y_train)

    # Predict on training
    y_train_pred = model.predict(X_train)
    print()
    print("For training set")
    print()
    get_metrics(y_train, y_train_pred)

    # Use model on validation set
    y_valid_pred = model.predict(X_valid)
    print()
    print("For validation set")
    print()
    get_metrics(y_valid, y_valid_pred)

    # Use model on test set
    y_test_pred = model.predict(X_test)
    print()
    print("For test set")
    print()
    get_metrics(y_test, y_test_pred)
    
    # Deleting unwanted variables
    del model
    del X_train
    del y_train
    del y_train_pred
    del y_valid_pred
    del y_test_pred
    gc.collect()
    
    print()
    print()
    print('#'*110)

##############################################################################################################

With sampling type: 1



For training set

Classification Report:                precision    recall  f1-score   support

           0       0.94      0.97      0.95      3480
           1       0.95      0.96      0.95      3480
           2       0.94      0.94      0.94      3480
           3       0.94      0.98      0.96      3480
           4       0.83      0.81      0.82      3480
           5       0.98      0.95      0.96      3480
           6       0.92      0.84      0.88      3480
           7       0.82      0.82      0.82      3480
           8       0.78      0.72      0.75      3480
           9       0.94      0.95      0.94      3480
          10       0.82      0.89      0.85      3480
          11       0.95      0.94      0.94      3480
          12       0.95      0.92      0.93      3480
          13       0.87      0.88      0.87      3480
          


For validation set

Classification Report:                precision    recall  f1-score   support

           0       0.83      0.83      0.83      2885
           1       0.78      0.74      0.76      1528
           2       0.86      0.86      0.86      9198
           3       0.77      0.80      0.79      1211
           4       0.71      0.72      0.71     11571
           5       0.88      0.72      0.79      1784
           6       0.68      0.52      0.59      1126
           7       0.63      0.49      0.55      4823
           8       0.60      0.74      0.66     23466
           9       0.76      0.62      0.68      1688
          10       0.62      0.54      0.58      4708
          11       0.79      0.65      0.72      1713
          12       0.85      0.64      0.73      1730
          13       0.63      0.56      0.60      4595
          14       0.66      0.58      0.61     13274

    accuracy                           0.68     85300
   macro avg       0.74      0.67  